In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import sys
from importlib import reload
from datetime import datetime
import tensorflow as tf
import time


# get the relative path
pref                = os.getcwd()

os.environ["RL_PATH"]   = pref

if f'{pref}/RLLibrary' not in sys.path:
    sys.path.append(f'{pref}/RLLibrary')


In [3]:
from RLLibrary.FinUseCases.PortfolioManagement import DataManager, EnvironmentManager
from RLLibrary.utils import constants

In [4]:
reload(DataManager)
reload(EnvironmentManager)

<module 'RLLibrary.FinUseCases.PortfolioManagement.EnvironmentManager' from '/Users/ankitgupta/Documents/git/anks/MachineLearning/ReinforcementLearning/RLLibrary/FinUseCases/PortfolioManagement/EnvironmentManager.py'>

In [8]:
assets = ["APA", "BMY"]
initialWeight = [0.5, 0.5]
actions = [
    [-0.03, 0.03], 
    [-0.02, 0.02], 
    [-0.01, 0.01], 
    [0.0, 0.0], 
    [0.01, -0.01], 
    [0.02, -0.02], 
    [0.03, -0.03], 
]

DATA_DIR = constants.DATA_DIR

env = EnvironmentManager.Portfolio_MultiStage(assets, initialWeight, \
                    nhistoricalDays = 5, \
                    startDate = "2019-01-01", endDate = "2019-12-31", \
                    actions = actions, \
                    assetDataPath = os.path.join(DATA_DIR, "PortfolioManagement"), \
                    config = {"initialCash": 1000000, "minCash": 0.02, "transactionFee": 0.0000}, \
                    penalizeFactors = {"Risk": -0.08, "ForbiddenAction": -8})

In [9]:
env.observation_space.currentState

(array([[[0.8807518 , 0.95138445, 0.91065977],
         [0.9176928 , 0.97294464, 0.94009104],
         [0.93745945, 0.99758479, 0.95549118],
         [0.95139339, 1.01298493, 0.98389582],
         [1.        , 1.0585008 , 1.00785157]],
 
        [[0.95633752, 1.08474034, 1.00911376],
         [0.99385337, 1.08314581, 1.04464917],
         [1.02607043, 1.11025298, 1.05603872],
         [1.01716826, 1.11344204, 1.07061732],
         [1.        , 1.09339641, 1.0646948 ]]]),
 array([0.4899998 , 0.49001075]),
 0.019989450000000002,
 [1, 1, 1, 1, 1, 1, 1])

In [10]:
next_state, reward, done, _ = env.step(actions[0])
reward

0.74

In [14]:
from RLLibrary.FinUseCases.PortfolioManagement import StrategyManager

In [17]:
from RLLibrary.FinUseCases.PortfolioManagement.ModelManager.A3C import Agent, Networks

In [32]:
reload(Agent)
reload(Networks)

<module 'RLLibrary.FinUseCases.PortfolioManagement.ModelManager.A3C.Networks' from '/Users/ankitgupta/Documents/git/anks/MachineLearning/ReinforcementLearning/RLLibrary/FinUseCases/PortfolioManagement/ModelManager/A3C/Networks.py'>

In [33]:
net1 = Networks.ActorCritic_CNN(nbHistory=env.nbHistory, action_size=len(actions))


In [34]:
probs, _value = net1(env.observation_space.currentState)


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.



/Users/ankitgupta/Documents/git/anks/MachineLearning/ReinforcementLearning/RLLibrary/FinUseCases/PortfolioManagement/ModelManager/A3C/Networks.py:114: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  if len(np.array(inputs).shape) == 1:


In [35]:
probs, _value

(<tf.Tensor: id=781, shape=(1, 7), dtype=float32, numpy=
 array([[0.08424149, 0.11817581, 0.1477979 , 0.18873116, 0.1857955 ,
         0.1140119 , 0.16124621]], dtype=float32)>,
 <tf.Tensor: id=925, shape=(1, 1), dtype=float32, numpy=array([[0.06925955]], dtype=float32)>)

In [36]:
net1.trainable_variables

[<tf.Variable 'actor_critic_cnn_3/conv2d_18/kernel:0' shape=(1, 3, 3, 2) dtype=float32, numpy=
 array([[[[ 0.08099592,  0.1868748 ],
          [ 0.1619997 ,  0.5966608 ],
          [-0.5802035 ,  0.33725786]],
 
         [[-0.31821728,  0.06201684],
          [ 0.17423236, -0.18781966],
          [-0.5456334 , -0.0275774 ]],
 
         [[-0.5382782 ,  0.6026153 ],
          [-0.15790746,  0.24871176],
          [-0.49932683, -0.5319152 ]]]], dtype=float32)>,
 <tf.Variable 'actor_critic_cnn_3/conv2d_18/bias:0' shape=(2,) dtype=float32, numpy=array([0., 0.], dtype=float32)>,
 <tf.Variable 'actor_critic_cnn_3/conv2d_19/kernel:0' shape=(1, 3, 2, 20) dtype=float32, numpy=
 array([[[[ 0.05870089, -0.08310859, -0.30029064, -0.20493342,
           -0.10290661, -0.11607353, -0.05815081,  0.05793327,
           -0.14832018,  0.0891355 , -0.0438318 ,  0.05990604,
            0.21816361, -0.09160544, -0.09452206, -0.19419691,
           -0.2892624 ,  0.05287632, -0.29903919,  0.1687597 ],
        

In [31]:
net1.actor.trainable_variables

AttributeError: 'Dummy' object has no attribute 'trainable_variables'